In [1]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

In [4]:
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'jQbipYrnhZmEqSHPg7kBO9v5j'
        consumer_secret = 'oMjU20nCRH1BISVkFVGf9uKWTmP9PKykEhJ7gdWPs7PBSuhp96'
        access_token = '172835284-O1EBpJwCWaXKlx27aMiFwDSi19kTsp5hpkJOChxy'
        access_token_secret = '20BAnIOOSpS305jd3wfkSsEbafFN5WRD6gVZztyvyfzWy'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 
  
def main(): 
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.get_tweets(query = 'Aam Aadmi Party', count = 2000) 
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    # print("Neutral tweets percentage: {} %".format(100*len(tweets - ntweets - ptweets)/len(tweets))) 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:200]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:200]: 
        print(tweet['text']) 
  
if __name__ == "__main__": 
    # calling main function 
    main() 

Positive tweets percentage: 23.076923076923077 %
Negative tweets percentage: 1.9230769230769231 %


Positive tweets:
RT @AAPTELANGANA: British Deputy High Commissioner to Telangana &amp; Andhra Pradesh, Dr. Andrew Fleming appreciates Aam Aadmi Party Political…
British Deputy High Commissioner to Telangana &amp; Andhra Pradesh, Dr. Andrew Fleming appreciates Aam Aadmi Party Poli… https://t.co/VIploDJBov
RT @AAPTELANGANA: "Street Vendor Act was passed in 2014 but it was not implemented till date. Delhi will be first state to implement it &amp; l…
RT @AAPTELANGANA: "We didnt give Street Vendors legal status, didnt organize them because of which all departments &amp; Policemen exploit them…
RT @AAPTELANGANA: "If we talk about Delhi, here also Street Vendors have an important role in the economy. If they dont work even for a sin…
"Street Vendor Act was passed in 2014 but it was not implemented till date. Delhi will be first state to implement… https://t.co/HVytM3x2vi
"We didnt give Street V